# ### 项目概述
# 本笔记本对S&P 500成分股的5年日级数据进行探索性分析，为后续的模型训练和策略回测提供数据洞察。
# 
# ### 分析目标
# 1. 理解数据结构和质量
# 2. 探索价格和交易量的分布特征
# 3. 分析股票间的相关性和波动性
# 4. 识别潜在的异常值和模式
# 5. 为特征工程提供依据

In [ ]:
# %% [markdown]
# ## 1. 环境配置和数据加载

# %%
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体和绘图样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")
sns.set_palette("husl")

# 设置图表大小
plt.rcParams['figure.figsize'] = [12, 8]

# 导入项目模块
import sys
sys.path.append('../src')
from data_preprocessing import DataPreprocessor

# %%
# 数据加载
print("加载数据...")
preprocessor = DataPreprocessor(data_path='../data/raw/all_stocks_5yr.csv')
raw_data = preprocessor.load_data()

print(f"数据形状: {raw_data.shape}")
print(f"数据列名: {raw_data.columns.tolist()}")

# %%
# 查看数据基本信息
print("=" * 60)
print("数据基本信息")
print("=" * 60)
print(f"时间范围: {raw_data['date'].min()} 到 {raw_data['date'].max()}")
print(f"股票数量: {raw_data['Name'].nunique()}")
print(f"总记录数: {len(raw_data)}")
print(f"平均每个股票的数据点: {len(raw_data) / raw_data['Name'].nunique():.0f}")

In [ ]:
# %% [markdown]
# ## 2. 数据质量检查

# %%
# 检查缺失值
print("检查缺失值...")
missing_data = raw_data.isnull().sum()
missing_percentage = (missing_data / len(raw_data)) * 100

missing_df = pd.DataFrame({
    '缺失数量': missing_data,
    '缺失百分比': missing_percentage
})
print(missing_df[missing_df['缺失数量'] > 0])

# %%
# 检查重复值
duplicates = raw_data.duplicated().sum()
print(f"重复记录数: {duplicates}")

# %%
# 检查数据完整性（每个股票的数据点数量）
stock_data_counts = raw_data['Name'].value_counts()
print("股票数据点数量统计:")
print(stock_data_counts.describe())

# 绘制股票数据点数量分布
plt.figure(figsize=(12, 6))
plt.hist(stock_data_counts.values, bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('数据点数量')
plt.ylabel('股票数量')
plt.title('每个股票的数据点数量分布')
plt.axvline(x=stock_data_counts.mean(), color='r', linestyle='--', label=f'平均: {stock_data_counts.mean():.0f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# %% [markdown]
# ## 3. 价格数据探索

# %%
# 提取价格数据的统计信息
price_columns = ['open', 'high', 'low', 'close']
price_stats = raw_data[price_columns].describe()
print("价格数据统计摘要:")
print(price_stats)

# %%
# 绘制价格分布图
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
price_columns = ['open', 'high', 'low', 'close']

for idx, col in enumerate(price_columns):
    ax = axes[idx // 2, idx % 2]
    
    # 原始价格分布
    ax.hist(raw_data[col].dropna(), bins=100, alpha=0.7, edgecolor='black', density=True)
    ax.set_xlabel(f'{col} 价格')
    ax.set_ylabel('密度')
    ax.set_title(f'{col} 价格分布')
    ax.grid(True, alpha=0.3)
    
    # 添加统计信息
    mean_price = raw_data[col].mean()
    median_price = raw_data[col].median()
    ax.axvline(x=mean_price, color='r', linestyle='--', label=f'均值: ${mean_price:.2f}')
    ax.axvline(x=median_price, color='g', linestyle='--', label=f'中位数: ${median_price:.2f}')
    ax.legend()

plt.tight_layout()
plt.show()

# %%
# 对数价格分布（金融数据通常取对数）
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for idx, col in enumerate(price_columns):
    ax = axes[idx // 2, idx % 2]
    
    # 对数价格
    log_prices = np.log(raw_data[col].dropna())
    
    ax.hist(log_prices, bins=100, alpha=0.7, edgecolor='black', density=True)
    ax.set_xlabel(f'log({col})')
    ax.set_ylabel('密度')
    ax.set_title(f'{col} 对数价格分布')
    ax.grid(True, alpha=0.3)
    
    # 添加正态分布拟合
    from scipy.stats import norm
    mu, std = norm.fit(log_prices)
    x = np.linspace(log_prices.min(), log_prices.max(), 100)
    p = norm.pdf(x, mu, std)
    ax.plot(x, p, 'r', linewidth=2, label=f'正态拟合\nμ={mu:.3f}, σ={std:.3f}')
    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
# %% [markdown]
# ## 4. 交易量数据分析

# %%
# 交易量数据探索
print("交易量数据统计:")
print(raw_data['volume'].describe())

# %%
# 交易量分布（使用对数尺度）
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 原始交易量分布
axes[0].hist(raw_data['volume'], bins=100, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('交易量')
axes[0].set_ylabel('频率')
axes[0].set_title('交易量分布（原始）')
axes[0].grid(True, alpha=0.3)

# 对数交易量分布
log_volume = np.log1p(raw_data['volume'])  # log(1+x) 处理零值
axes[1].hist(log_volume, bins=100, alpha=0.7, edgecolor='black', color='orange')
axes[1].set_xlabel('log(1+交易量)')
axes[1].set_ylabel('频率')
axes[1].set_title('交易量对数分布')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %%
# 价格与交易量的关系
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for idx, price_col in enumerate(price_columns):
    ax = axes[idx // 2, idx % 2]
    
    # 采样以加快绘图速度
    sample_size = min(10000, len(raw_data))
    sample_idx = np.random.choice(len(raw_data), sample_size, replace=False)
    
    ax.scatter(raw_data[price_col].iloc[sample_idx], 
               np.log1p(raw_data['volume'].iloc[sample_idx]), 
               alpha=0.3, s=10)
    ax.set_xlabel(f'{price_col} 价格')
    ax.set_ylabel('log(1+交易量)')
    ax.set_title(f'{price_col} vs 交易量')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# %% [markdown]
# ## 5. 时间序列分析

# %%
# 选择几只代表性股票进行分析
sample_stocks = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']

# 筛选数据
sample_data = raw_data[raw_data['Name'].isin(sample_stocks)].copy()
sample_data['date'] = pd.to_datetime(sample_data['date'])

# %%
# 绘制股价时间序列
fig, axes = plt.subplots(len(sample_stocks), 1, figsize=(15, 3*len(sample_stocks)))

for idx, stock in enumerate(sample_stocks):
    stock_data = sample_data[sample_data['Name'] == stock].sort_values('date')
    
    axes[idx].plot(stock_data['date'], stock_data['close'], linewidth=1)
    axes[idx].set_title(f'{stock} 收盘价时间序列')
    axes[idx].set_xlabel('日期')
    axes[idx].set_ylabel('收盘价 ($)')
    axes[idx].grid(True, alpha=0.3)
    
    # 添加移动平均线
    stock_data['sma_50'] = stock_data['close'].rolling(window=50).mean()
    axes[idx].plot(stock_data['date'], stock_data['sma_50'], 'r--', linewidth=1, label='50日均线')
    
    axes[idx].legend()

plt.tight_layout()
plt.show()

# %%
# 计算日收益率
def calculate_returns(prices):
    return prices.pct_change()

# 为样本股票计算收益率
returns_data = pd.DataFrame()

for stock in sample_stocks:
    stock_data = sample_data[sample_data['Name'] == stock].sort_values('date')
    returns_data[stock] = calculate_returns(stock_data['close']).values

returns_data = returns_data.dropna()

# %%
# 收益率统计
print("收益率统计摘要:")
print(returns_data.describe())

# %%
# 收益率分布图
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, stock in enumerate(sample_stocks):
    ax = axes[idx]
    
    returns = returns_data[stock].dropna()
    
    # 直方图
    ax.hist(returns, bins=100, alpha=0.7, edgecolor='black', density=True)
    ax.set_xlabel('日收益率')
    ax.set_ylabel('密度')
    ax.set_title(f'{stock} 收益率分布')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-0.1, 0.1)
    
    # 添加正态分布拟合
    from scipy.stats import norm
    mu, std = norm.fit(returns)
    x = np.linspace(returns.min(), returns.max(), 100)
    p = norm.pdf(x, mu, std)
    ax.plot(x, p, 'r', linewidth=2, label=f'μ={mu:.4f}, σ={std:.4f}')
    ax.legend()

# 移除多余的子图
for i in range(len(sample_stocks), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 6. 相关性分析

# %%
# 计算收益率的相关性矩阵
correlation_matrix = returns_data.corr()

print("收益率相关性矩阵:")
print(correlation_matrix)

# %%
# 相关性热力图
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('股票收益率相关性热力图')
plt.tight_layout()
plt.show()

# %%
# 价格水平的相关性
# 计算收盘价的相关性（需要对齐时间）
pivot_data = sample_data.pivot_table(index='date', columns='Name', values='close')
price_correlation = pivot_data.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(price_correlation, annot=True, cmap='viridis', center=0.5,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('股票价格相关性热力图')
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 7. 波动性分析

# %%
# 计算滚动波动率（年化）
def calculate_volatility(returns, window=20):
    return returns.rolling(window=window).std() * np.sqrt(252)

# 为样本股票计算波动率
volatility_data = pd.DataFrame()

for stock in sample_stocks:
    returns = returns_data[stock]
    volatility_data[stock] = calculate_volatility(returns, window=20)

volatility_data = volatility_data.dropna()

# %%
# 波动率时间序列
fig, axes = plt.subplots(len(sample_stocks), 1, figsize=(15, 3*len(sample_stocks)))

for idx, stock in enumerate(sample_stocks):
    axes[idx].plot(volatility_data.index, volatility_data[stock], linewidth=1)
    axes[idx].set_title(f'{stock} 滚动年化波动率 (20天窗口)')
    axes[idx].set_xlabel('时间')
    axes[idx].set_ylabel('波动率')
    axes[idx].grid(True, alpha=0.3)
    
    # 添加平均波动率线
    avg_vol = volatility_data[stock].mean()
    axes[idx].axhline(y=avg_vol, color='r', linestyle='--', label=f'平均: {avg_vol:.3f}')
    axes[idx].legend()

plt.tight_layout()
plt.show()

# %%
# 波动率分布比较
plt.figure(figsize=(12, 6))
volatility_data.boxplot()
plt.title('股票波动率分布比较')
plt.ylabel('年化波动率')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# %% [markdown]
# ## 8. 市场整体分析

# %%
# 计算市场指数（等权重投资组合）
market_returns = returns_data.mean(axis=1)

print("市场指数（等权重组合）收益率统计:")
print(market_returns.describe())

# %%
# 市场指数累计收益率
cumulative_market_return = (1 + market_returns).cumprod()

plt.figure(figsize=(12, 6))
plt.plot(cumulative_market_return.index, cumulative_market_return.values, linewidth=2)
plt.title('市场指数累计收益率（等权重组合）')
plt.xlabel('时间')
plt.ylabel('累计收益倍数')
plt.grid(True, alpha=0.3)
plt.show()

# %%
# 市场波动率随时间变化
market_volatility = calculate_volatility(market_returns, window=60)

plt.figure(figsize=(12, 6))
plt.plot(market_volatility.index, market_volatility.values, linewidth=2, color='orange')
plt.fill_between(market_volatility.index, 0, market_volatility.values, alpha=0.3, color='orange')
plt.title('市场滚动波动率 (60天窗口)')
plt.xlabel('时间')
plt.ylabel('年化波动率')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# %% [markdown]
# ## 9. 异常值检测

# %%
# 检测极端收益率
extreme_threshold = 3  # 3倍标准差

extreme_events = {}
for stock in sample_stocks:
    returns = returns_data[stock]
    mean_return = returns.mean()
    std_return = returns.std()
    
    extreme_idx = np.where(np.abs(returns - mean_return) > extreme_threshold * std_return)[0]
    extreme_events[stock] = {
        'count': len(extreme_idx),
        'percentage': len(extreme_idx) / len(returns) * 100,
        'max_return': returns.max(),
        'min_return': returns.min()
    }

# 显示极端事件统计
extreme_df = pd.DataFrame(extreme_events).T
print("极端收益率事件统计 (±3σ):")
print(extreme_df)

# %%
# 可视化极端收益率
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, stock in enumerate(sample_stocks):
    ax = axes[idx]
    
    returns = returns_data[stock]
    
    # 绘制收益率时间序列
    ax.plot(returns.index, returns.values, linewidth=0.5, alpha=0.7)
    
    # 标记极端事件
    mean_return = returns.mean()
    std_return = returns.std()
    upper_threshold = mean_return + extreme_threshold * std_return
    lower_threshold = mean_return - extreme_threshold * std_return
    
    extreme_upper = returns[returns > upper_threshold]
    extreme_lower = returns[returns < lower_threshold]
    
    ax.scatter(extreme_upper.index, extreme_upper.values, color='red', s=20, label='极端正收益')
    ax.scatter(extreme_lower.index, extreme_lower.values, color='blue', s=20, label='极端负收益')
    
    ax.set_title(f'{stock} 极端收益率检测')
    ax.set_xlabel('时间')
    ax.set_ylabel('日收益率')
    ax.grid(True, alpha=0.3)
    ax.legend()

# 移除多余的子图
for i in range(len(sample_stocks), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 10. 技术指标探索

# %%
# 计算技术指标示例（以AAPL为例）
aapl_data = sample_data[sample_data['Name'] == 'AAPL'].sort_values('date').copy()

# 移动平均线
aapl_data['SMA_20'] = aapl_data['close'].rolling(window=20).mean()
aapl_data['SMA_50'] = aapl_data['close'].rolling(window=50).mean()

# 布林带
aapl_data['BB_middle'] = aapl_data['close'].rolling(window=20).mean()
aapl_data['BB_std'] = aapl_data['close'].rolling(window=20).std()
aapl_data['BB_upper'] = aapl_data['BB_middle'] + 2 * aapl_data['BB_std']
aapl_data['BB_lower'] = aapl_data['BB_middle'] - 2 * aapl_data['BB_std']

# RSI
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

aapl_data['RSI'] = calculate_rsi(aapl_data['close'], window=14)

# %%
# 技术指标可视化
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 1. 价格与移动平均线
ax1 = axes[0]
ax1.plot(aapl_data['date'], aapl_data['close'], label='收盘价', linewidth=1, color='black')
ax1.plot(aapl_data['date'], aapl_data['SMA_20'], label='20日移动平均', linewidth=1, color='blue')
ax1.plot(aapl_data['date'], aapl_data['SMA_50'], label='50日移动平均', linewidth=1, color='red')
ax1.fill_between(aapl_data['date'], aapl_data['BB_upper'], aapl_data['BB_lower'], 
                 alpha=0.2, color='gray', label='布林带')
ax1.set_title('AAPL: 价格与移动平均线')
ax1.set_xlabel('日期')
ax1.set_ylabel('价格 ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. 布林带
ax2 = axes[1]
ax2.plot(aapl_data['date'], aapl_data['close'], label='收盘价', linewidth=1, color='black')
ax2.plot(aapl_data['date'], aapl_data['BB_middle'], label='中轨 (20日SMA)', linewidth=1, color='blue')
ax2.plot(aapl_data['date'], aapl_data['BB_upper'], label='上轨 (+2σ)', linewidth=1, color='red', linestyle='--')
ax2.plot(aapl_data['date'], aapl_data['BB_lower'], label='下轨 (-2σ)', linewidth=1, color='green', linestyle='--')
ax2.fill_between(aapl_data['date'], aapl_data['BB_upper'], aapl_data['BB_lower'], 
                 alpha=0.2, color='gray')
ax2.set_title('AAPL: 布林带指标')
ax2.set_xlabel('日期')
ax2.set_ylabel('价格 ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. RSI指标
ax3 = axes[2]
ax3.plot(aapl_data['date'], aapl_data['RSI'], linewidth=1, color='purple')
ax3.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='超买线 (70)')
ax3.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='超卖线 (30)')
ax3.axhline(y=50, color='gray', linestyle='-', alpha=0.5, label='中线 (50)')
ax3.fill_between(aapl_data['date'], 70, 100, alpha=0.2, color='red')
ax3.fill_between(aapl_data['date'], 0, 30, alpha=0.2, color='green')
ax3.set_title('AAPL: RSI指标')
ax3.set_xlabel('日期')
ax3.set_ylabel('RSI值')
ax3.set_ylim(0, 100)
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# ## 11. 特征相关性分析

# %%
# 使用预处理模块计算技术指标
print("使用预处理模块计算技术指标...")
preprocessor = DataPreprocessor(data_path='../data/raw/all_stocks_5yr.csv')
raw_data = preprocessor.load_data()
processed_data = preprocessor.clean_data()

# 选择AAPL数据进行分析
aapl_processed = processed_data[processed_data['Name'] == 'AAPL'].copy()
aapl_processed['date'] = pd.to_datetime(aapl_processed['date'])

# 选择数值特征进行分析
numeric_features = ['open', 'high', 'low', 'close', 'volume', 
                    'daily_return', 'log_return', 'sma_5', 'sma_10', 
                    'sma_20', 'bb_width', 'rsi', 'macd', 'volatility']

# 计算特征相关性矩阵
correlation_matrix = aapl_processed[numeric_features].corr()

# %%
# 特征相关性热力图
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('AAPL技术特征相关性热力图')
plt.tight_layout()
plt.show()

# %%
# 目标变量（未来收益率）与特征的相关性
# 这里我们使用下一天的收益率作为目标
aapl_processed['future_return'] = aapl_processed['daily_return'].shift(-1)
target_correlation = aapl_processed[numeric_features + ['future_return']].corr()['future_return'].drop('future_return')

# 排序并显示
target_correlation_sorted = target_correlation.sort_values(ascending=False)

print("特征与未来收益率的相关性:")
print(target_correlation_sorted)

# %%
# 可视化特征与目标的相关性
plt.figure(figsize=(12, 6))
bars = plt.bar(range(len(target_correlation_sorted)), target_correlation_sorted.values)
plt.xticks(range(len(target_correlation_sorted)), target_correlation_sorted.index, rotation=45, ha='right')
plt.xlabel('特征')
plt.ylabel('与未来收益率的相关系数')
plt.title('特征与未来收益率的相关性')
plt.grid(True, alpha=0.3, axis='y')

# 颜色编码：正相关为绿色，负相关为红色
for i, bar in enumerate(bars):
    if target_correlation_sorted.values[i] > 0:
        bar.set_color('green')
    else:
        bar.set_color('red')

plt.tight_layout()
plt.show()


In [ ]:
# %% [markdown]
# ## 12. 总结与结论

# %%
# 生成EDA总结报告
print("=" * 60)
print("EDA分析总结报告")
print("=" * 60)

print("\n1. 数据质量:")
print(f"   - 总记录数: {len(raw_data)}")
print(f"   - 股票数量: {raw_data['Name'].nunique()}")
print(f"   - 缺失值比例: {(raw_data.isnull().sum().sum() / (len(raw_data) * len(raw_data.columns)) * 100):.2f}%")
print(f"   - 时间范围: {raw_data['date'].min()} 到 {raw_data['date'].max()}")

print("\n2. 价格特征:")
print(f"   - 平均收盘价: ${raw_data['close'].mean():.2f}")
print(f"   - 价格标准差: ${raw_data['close'].std():.2f}")
print(f"   - 价格偏度: {raw_data['close'].skew():.3f} (正偏，存在高价值股票)")
print(f"   - 价格峰度: {raw_data['close'].kurtosis():.3f} (尖峰，极端值较多)")

print("\n3. 收益率特征:")
print(f"   - 平均日收益率: {returns_data.mean().mean():.4f}")
print(f"   - 收益率标准差: {returns_data.std().mean():.4f}")
print(f"   - 收益率偏度: {returns_data.skew().mean():.3f} (通常为负，左偏)")
print(f"   - 收益率峰度: {returns_data.kurtosis().mean():.3f} (尖峰，厚尾分布)")

print("\n4. 市场特征:")
print(f"   - 市场平均波动率: {market_volatility.mean():.4f}")
print(f"   - 最大回撤观察: 需进一步分析")
print(f"   - 股票间平均相关性: {correlation_matrix.values[np.triu_indices_from(correlation_matrix, k=1)].mean():.3f}")

print("\n5. 异常检测:")
for stock, stats in extreme_events.items():
    print(f"   - {stock}极端收益率事件: {stats['count']}次 ({stats['percentage']:.2f}%)")

print("\n6. 建模建议:")
print("   - 特征工程: 使用对数收益率、技术指标、波动率特征")
print("   - 数据标准化: 建议使用StandardScaler")
print("   - 时间序列分割: 按时间顺序划分训练集和测试集")
print("   - 模型选择: LSTM和Transformer适合捕捉时间依赖")
print("   - 风险管理: 需要考虑止损止盈和仓位控制")

# %%
# 保存EDA结果到文件
import os
os.makedirs('../results/eda', exist_ok=True)

# 保存关键数据
key_stats = {
    '数据质量': {
        '总记录数': len(raw_data),
        '股票数量': raw_data['Name'].nunique(),
        '时间范围': f"{raw_data['date'].min()} 到 {raw_data['date'].max()}"
    },
    '价格统计': {
        '平均收盘价': raw_data['close'].mean(),
        '收盘价标准差': raw_data['close'].std(),
        '收盘价偏度': raw_data['close'].skew(),
        '收盘价峰度': raw_data['close'].kurtosis()
    },
    '收益率统计': {
        '平均日收益率': returns_data.mean().mean(),
        '收益率标准差': returns_data.std().mean(),
        '收益率偏度': returns_data.skew().mean(),
        '收益率峰度': returns_data.kurtosis().mean()
    },
    '市场特征': {
        '平均波动率': market_volatility.mean(),
        '股票间平均相关性': correlation_matrix.values[np.triu_indices_from(correlation_matrix, k=1)].mean()
    }
}

# 保存为JSON文件
import json
with open('../results/eda/eda_summary.json', 'w') as f:
    json.dump(key_stats, f, indent=4, default=str)

print(f"\nEDA总结已保存到: ../results/eda/eda_summary.json")